In [1]:
import pandas_datareader as pdr
from datetime import datetime
import json

In [2]:
start_date = datetime(1970, 1, 1)
end_date = datetime.now().date()
tickers_path = 'data/tickers/sp500.json'
dst_path = 'data/stocks/'

In [20]:
tickers_path = 'data/tickers/test.json'
dst_path = 'data/stocks/test/'

In [31]:
def download_data(tickers_path, dst_path, start_date=datetime(1970, 1, 1), end_date=datetime.now().date(), columns=['Close']):

    assert 'Close' in columns

    with open(tickers_path, 'r') as f:
        tickers = json.load(f)

    success = []
    fail = []

    for ticker in tickers:
        
        ticker = ticker.replace(".", "-")

        try:
            df = pdr.get_data_yahoo(symbols=ticker, start=start_date, end=end_date)

            drop_list = list(set(df.columns) - set(columns))
            df.drop(drop_list,axis=1,inplace=True)
            
            print('ok')


            download_path = dst_path + ticker + '.csv'
            df.to_csv(download_path, index=True)

            success.append(ticker)
        except:
            fail.append(ticker)

    return success, fail

In [32]:
success, fail = download_data(tickers_path, dst_path)

In [26]:
fail

['A', 'AAL']